In [2]:
#this is a script for compiling data that has been fully parallelized on the cluster. 

import pandas as pd
import matplotlib.pyplot as plt
import os
import string

In [3]:
cwd = os.getcwd()
print(cwd)

/Users/mayachari/Desktop/bird/Parallel_Hoffman_full


In [5]:
target_file_get = os.getcwd() + "/"+ "5023583" +"_output_files/"+  "10_4out.csv"
pd.read_csv(target_file_get)

#look at one mpd randomization to see data structure. 

,Unnamed: 0,plot,ntaxa,mpd,mpd.rnd,mpd.sd,nri,mpd.ranklow,mpd.rankhi,mntd,mntd.rnd,mntd.sd,nti,mntd.ranklo,mntd.rankhi,runs
0,1,species,10,334.6871,251.2945,NaN,NaN,0,1,239.1375,182.9458,NaN,NaN,0,1,1


In [7]:
tree_sizes = [10,50,100,200,1000] # define the tree sizes that we have data for (aka: parameter combinations)



In [ ]:
num_files = 1; #predetermine the number of files/runs. This is equivalent to the array_ID from the r script. 
for i in tree_sizes: 